# Setup & Library Import

In [10]:
import pandas as pd
from datetime import datetime
import numpy as np

import stumpy

In [26]:
FP_NO_ANOMALY = "../data/art_noisy.csv"
FP_ANOMALY = "../data/art_increase_spike_density.csv"
FP_TAXI = "../data/nyc_taxi.csv"

In [4]:
def time_log(f):
    def wrapper(*args, **kwargs):
        tick = datetime.now()
        results = f(*args, **kwargs)
        tock = datetime.now()
        print(f"{f.__name__} function took {tock-tick}")
        return results
    wrapper.unwrapped = f
    return wrapper

# Load DataFrame

In [5]:
@time_log
def read_dataframe(filepath: str) -> pd.DataFrame():
    """This function read dataframe 

    Args:
        filepath (str): the file_path directory

    Returns:
        df (pd.DataFrame): Time-series DataFrame
    """
    df = (pd.read_csv(filepath, parse_dates=True)
          .assign(timestamp = lambda x: pd.to_datetime(x.timestamp))
          .set_index(["timestamp"])
          .sort_index()
          )
    return df

In [6]:
df = (
    pd.concat(
        [
            read_dataframe(FP_ANOMALY).rename(
                columns={"value": "Art Store 1"}
            ),
            read_dataframe(FP_NO_ANOMALY).rename(
                columns={"value": "Art Store 2"}
            )
        ],
        axis=1
    )
)
df.head()

read_dataframe function took 0:00:00.083201
read_dataframe function took 0:00:00.003852


,Art Store 1,Art Store 2
timestamp,,
2014-04-01 00:00:00,20.0,18.622185
2014-04-01 00:05:00,0.0,8.163417
2014-04-01 00:10:00,0.0,13.292383
2014-04-01 00:15:00,0.0,11.667046
2014-04-01 00:20:00,0.0,12.940358


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4032 entries, 2014-04-01 00:00:00 to 2014-04-14 23:55:00
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Art Store 1  4032 non-null   float64
 1   Art Store 2  4032 non-null   float64
dtypes: float64(2)
memory usage: 94.5 KB


# Define Matrix Profile & Normalized Function

In [22]:
def normalize(df: pd.DataFrame) -> pd.DataFrame():
    """This function normalize the time-series dataframe

    Args:
        df (pd.DataFrame): Time-series dataframe

    Returns:
        df (pd.DataFrame): Time-series dataframe normalized.
    """
    df = ((df - df.min()) / (df.max() - df.min()))
    return df


@time_log
def anomaly_detection_matrix_profile(df, window_size: int=96, smooth_n: int=10, normalize_data: bool=True,
                                     plot_results: bool=True):
    """This function calculate the Matrix Profile using the STUMPY library

    Args:
        df (_type_): Time-Series data frame
        window_size (int, optional): Approximately, how many data points might be found in a pattern. Defaults to 96.
        smooth_n (int, optional): Time windows to smoot matrix profile scores. Defaults to 10.
        normalize_data (bool, optional): Normalize dataframe or not. Defaults to True.
        plot_results (bool, optional): Return a plotly plot of the results, including the original time-series and the matrix profile. Defaults to True.
    Returns:
        matrix_profile (dict): Matrix Profile Scores
        df (pd.DataFrame): Dataframe ranking each column/feature overall (e.g st = largest anomalies (useful when input df has multiple columns))
    """
    scores = {}
    matrix_profile_dists = {}

    if (len(df.shape) <= 1):
        df = df.to_frame()
    else:
        pass

    if (normalize_data):
        df = (normalize(df).dropna())
    else:
        pass

    for c in df.columns:
        matrix_profile = stumpy.stump(df[c].astype("float"), m=window_size,
                          ignore_trivial=True, normalize=False)
        matrix_profile_dist = matrix_profile[:, 0]

        # score the matrix profile based on distance between the 99.5th percentile and
        # 75th percentile for the chosen smooth window
        df_mp_dist = pd.DataFrame(matrix_profile_dist).rolling(smooth_n).mean()
        scores[c] = df_mp_dist[0].quantile(0.995) - df_mp_dist[0].quantile(0.75)

        # fill nans
        nan_value_count = np.empty(len(df[c]) - len(matrix_profile_dist))
        nan_value_count.fill(np.nan)
        matrix_profile_dist = np.concatenate((nan_value_count, matrix_profile_dist.astype(float)))

        # record
        matrix_profile_dists[c] = matrix_profile_dist

    df_scores = pd.DataFrame.from_dict(scores, orient="index", columns=["score"])
    df_scores["rank"] = df_scores["score"].rank(ascending=False)

    if (plot_results):
        pd.options.plotting.backend = "plotly"

        for c in df_scores.sort_values("rank").index:
            score = round(df_scores.loc[c]["score"], 4)
            rank = df_scores.loc[c]["rank"]
            df_tmp = df[[c]].assign(MatrixProfileScore=matrix_profile_dists[c])
            fig = df_tmp.plot(template="simple_white", title=f"{c}")
            fig.show()
    else:
        pass
    
    return matrix_profile_dists, df_scores.sort_values("rank")

In [23]:
matrix_profile_dists, scores_df = (
    anomaly_detection_matrix_profile(
        df, window_size=30,
        smooth_n=int(5*30),
        normalize_data=True,
        plot_results=True
    )
)

anomaly_detection_matrix_profile function took 0:00:00.367249


The plots output shows that the "Art Store 1"is the worst offender.

To better understand, the Matrix Profile (MP) should be interpreted in its own right and on it's own scale. Looking for deviations from the norm and not the values themselves. So for example, for "Art Store 2" the MP stays within defined bounds, that signifying no anomalies. In contrast, the MP for "Art Store 1" shoots up dramatically at the point of an anomaly.

In [24]:
scores_df

,score,rank
Art Store 1,0.347509,1.0
Art Store 2,0.082621,2.0


In [25]:
matrix_profile_dists

{'Art Store 1': array([nan, nan, nan, ...,  0.,  0.,  0.]),
 'Art Store 2': array([       nan,        nan,        nan, ..., 1.31372639, 1.21523906,
        1.28573455])}

# Apply for NYC TAXI Demand

In [27]:
df = read_dataframe(FP_TAXI)
df.head()

read_dataframe function took 0:00:00.018256


,value
timestamp,
2014-07-01 00:00:00,10844
2014-07-01 00:30:00,8127
2014-07-01 01:00:00,6210
2014-07-01 01:30:00,4656
2014-07-01 02:00:00,3820


# Run Matrix Profile

In [28]:
matrix_profile_dists, scores_df = (
    anomaly_detection_matrix_profile(
        df, window_size=30,
        smooth_n=int(5*30),
        normalize_data=True,
        plot_results=True
    )
)

anomaly_detection_matrix_profile function took 0:00:00.751489


The algorithm result shows that it not only picks up exceptionally high demand, but also low demand. So we can assign the MP directly to the original dataframe. 

NOTE: We can play around with various threshold to determine what you classify as an anomaly.


In [29]:
matrix_profile_dists

{'value': array([       nan,        nan,        nan, ..., 0.14008596, 0.14475898,
        0.15285224])}

In [33]:
df = (
    df
    .assign(MatrixProfile=pd.Series(matrix_profile_dists["value"]).fillna(0.0).values)
)

In [34]:
df.head()

,value,MatrixProfile
timestamp,,
2014-07-01 00:00:00,10844,0.0
2014-07-01 00:30:00,8127,0.0
2014-07-01 01:00:00,6210,0.0
2014-07-01 01:30:00,4656,0.0
2014-07-01 02:00:00,3820,0.0


In [35]:
df.tail()

,value,MatrixProfile
timestamp,,
2015-01-31 21:30:00,24670,0.139677
2015-01-31 22:00:00,25721,0.139834
2015-01-31 22:30:00,27309,0.140086
2015-01-31 23:00:00,26591,0.144759
2015-01-31 23:30:00,26288,0.152852
